In [1]:
import warnings

import pandas as pd

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [2]:
"""2022, 2023 Data concat"""
path2022 = "./data/train_2022"
visit_area_info2022 = pd.read_csv(path2022 + "/tn_visit_area_info_방문지정보_B.csv")  # 방문지 정보 activity
travel2022 = pd.read_csv(path2022 + "/tn_travel_여행_B.csv")  # 여행 travel
traveller_master2022 = pd.read_csv(path2022 + "/tn_traveller_master_여행객 Master_B.csv")  # 여행객 정보 Master traveler

path2023 = "./data/train_2023"
visit_area_info2023 = pd.read_csv(path2023 + "/tn_visit_area_info_방문지정보_F.csv")  # 방문지 정보 activity
travel2023 = pd.read_csv(path2023 + "/tn_travel_여행_F.csv")  # 여행 travel
traveller_master2023 = pd.read_csv(path2023 + "/tn_traveller_master_여행객 Master_F.csv")  # 여행객 정보 Master traveler

In [3]:
"""Data concat"""
visi_area_info = pd.concat([visit_area_info2022, visit_area_info2023], ignore_index=True)
travel = pd.concat([travel2022, travel2023], ignore_index=True)
traveller_master = pd.concat([traveller_master2022, traveller_master2023], ignore_index=True)

In [4]:
"""Visit area information"""

'Visit area information'

In [5]:
def visit_area_information(visit_area_info):
    """방문지 중 유의미한 정보만 추출(1~8까지는 관광지, 나머지는 무의미한 정보)"""
    valid_types = range(1, 9)
    visit_area_info = visit_area_info[visit_area_info["VISIT_AREA_TYPE_CD"].isin(valid_types)]
    visit_area_info = visit_area_info.dropna(subset=["LOTNO_ADDR"])
    visit_area_info = visit_area_info.reset_index(drop=True)
    return visit_area_info

visit_area_spot_info= visit_area_information(visi_area_info)

In [6]:
def address_info(visit_area_info):
    """주소에서 시도와 군구 추출"""
    sido = []
    gungu = []
    for i in range(len(visit_area_info['LOTNO_ADDR'])):
        sido.append(visit_area_info['LOTNO_ADDR'][i].split(' ')[0])
        gungu.append(visit_area_info['LOTNO_ADDR'][i].split(' ')[1])
    return sido, gungu

sido, gungu = address_info(visit_area_spot_info)

visit_area_spot_info['SIDO'] = sido
visit_area_spot_info['GUNGU'] = gungu

busan_spot_info = visit_area_spot_info[visit_area_spot_info['SIDO'] == "부산"]

In [7]:
"""
유의미한 변수만 추출
여행ID 방문지명 시도 군구 방문지유형코드 만족도 재방문의향 추천의향 체류시간(분) 재방문여부"
"""
busan_spot_info = busan_spot_info[['TRAVEL_ID', 'VISIT_AREA_NM', 'SIDO', 'GUNGU', 'VISIT_AREA_TYPE_CD', 'DGSTFN',
                                  'REVISIT_INTENTION', 'RCMDTN_INTENTION', 'RESIDENCE_TIME_MIN', 'REVISIT_YN']].reset_index(drop=True)


In [8]:
def mission_check(travel):
    travel_list = []
    for i in range(len(travel)):
        value = int(travel['TRAVEL_MISSION_CHECK'][i].split(';')[0])
        travel_list.append(value)
    return travel_list


travel['TRAVEL_MISSION_PRIORITY'] = mission_check(travel)
travel = travel[['TRAVEL_ID', 'TRAVELER_ID', 'TRAVEL_MISSION_PRIORITY']]

In [9]:
traveller_master = traveller_master[['TRAVELER_ID', 'GENDER', 'AGE_GRP', 'INCOME', 'TRAVEL_STYL_1', 
                                     'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 
                                     'TRAVEL_STYL_6', 'TRAVEL_STYL_7','TRAVEL_STYL_8', 
                                      'TRAVEL_MOTIVE_1', 'TRAVEL_NUM', 'TRAVEL_COMPANIONS_NUM' ]]



In [10]:
"""busan spot info와 traveller_master, travel merge Result: 부산에 여행온 사람들의 정보"""
df = pd.merge(travel, traveller_master, left_on = 'TRAVELER_ID', right_on = 'TRAVELER_ID', how = 'inner')
df = pd.merge(busan_spot_info, df, left_on = 'TRAVEL_ID', right_on = 'TRAVEL_ID', how = 'left')  

In [12]:
df

,TRAVEL_ID,VISIT_AREA_NM,SIDO,GUNGU,VISIT_AREA_TYPE_CD,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION,RESIDENCE_TIME_MIN,REVISIT_YN,TRAVELER_ID,TRAVEL_MISSION_PRIORITY,GENDER,AGE_GRP,INCOME,TRAVEL_STYL_1,TRAVEL_STYL_2,TRAVEL_STYL_3,TRAVEL_STYL_4,TRAVEL_STYL_5,TRAVEL_STYL_6,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_MOTIVE_1,TRAVEL_NUM,TRAVEL_COMPANIONS_NUM
0,b_b001138,광안리해수욕장,부산,수영구,1,4.0,4.0,4.0,30.0,N,b001138,4.0,남,50.0,7.0,6.0,2.0,2.0,4.0,6.0,6.0,2.0,6.0,4.0,3.0,0.0
1,b_b002682,센텀시티 스파랜드,부산,해운대구,6,3.0,2.0,4.0,60.0,N,b002682,22.0,남,20.0,4.0,4.0,2.0,4.0,3.0,6.0,4.0,6.0,6.0,1.0,2.0,0.0
2,b_b008221,F1963,부산,수영구,4,4.0,3.0,3.0,90.0,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,b_b010232,흰여울문화마을,부산,영도구,7,5.0,5.0,5.0,30.0,N,b010232,1.0,여,20.0,3.0,5.0,3.0,4.0,6.0,4.0,4.0,2.0,7.0,5.0,5.0,0.0
4,b_b000618,베스트루이스해밀턴호텔 오션테라스,부산,기장군,6,3.0,2.0,3.0,150.0,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3234,f_f016808,BIFF 광장,부산,중구,4,4.0,4.0,4.0,30.0,Y,f016808,1.0,남,40.0,2.0,2.0,2.0,1.0,6.0,2.0,5.0,3.0,7.0,2.0,2.0,2.0
3235,f_f016808,꽃분이네,부산,중구,2,3.0,2.0,3.0,30.0,N,f016808,1.0,남,40.0,2.0,2.0,2.0,1.0,6.0,2.0,5.0,3.0,7.0,2.0,2.0,2.0
3236,f_f016808,남포지하쇼핑센터,부산,중구,4,4.0,4.0,4.0,60.0,Y,f016808,1.0,남,40.0,2.0,2.0,2.0,1.0,6.0,2.0,5.0,3.0,7.0,2.0,2.0,2.0
3237,g_g000007,송정해수욕장,부산,해운대구,1,5.0,5.0,5.0,60.0,N,g000007,4.0,남,20.0,4.0,7.0,1.0,1.0,5.0,1.0,1.0,7.0,1.0,2.0,1.0,0.0
